use simplified data 1

data source:

data v2 取用藥1-5 款, 刪除全0 col, 處方&針炙ignored

In [25]:
import numpy as np
import pandas as pd
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy



In [59]:
def ReadData(FILENAME):
    data = pd.read_csv(FILENAME)
    print("in ReadData")
    print("type of data:", type(data))
    print(f'Shape of data = ({data.shape[0]} rows, {data.shape[1]} cols).')
    return data

def SplitXY(data, data_left_bound,label_left_bound):
    """label_left_bound: 第一個藥材的col no."""
    # Body status: 1~3, Diagnosis: 4~7, Symptom: 11~124
    # Prescription: 125~226
    # split_X = list(range(0, label_left_bound))
    # split_Y = list(range(label_left_bound, len(data.columns)))
    X = data.iloc[1:, list(range(data_left_bound, label_left_bound))]
    y = data.iloc[1:, list(range(label_left_bound, len(data.columns)))]
    # Debug
    print("SplitXY:")
    print(f'Shape of X = ({X.shape[0]} rows, {X.shape[1]} cols). First 10 data of X:')
    print(X.iloc[:10, :10])
    print(f'Shape of y = ({y.shape[0]} rows, {y.shape[1]} cols). First 10 data of y:')
    print(y.iloc[:10, :10])
    return X, y

In [41]:
data_pd = ReadData("./simplified_data/simplified_1.csv")

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (113 rows, 182 cols).


In [46]:
print(data_pd.columns[102])
# 麻黃 是第一個藥材,在第109col
print(data_pd.iloc[0, :2])

麻黃
Unnamed: 0    3
炙甘草           0
Name: 0, dtype: int64


In [71]:
X,y = SplitXY(data_pd, 1, 102)
X_np = X.values.astype('float64')
y_np = y.values.astype('float64')


SplitXY:
Shape of X = (112 rows, 101 cols). First 10 data of X:
    炙甘草  乳癌  肺癌  肝癌  腺癌  攝護腺癌  骨癌  腦瘤  肝炎  回診
1     0   0   0   0   0     0   0   0   0   1
2     0   0   0   0   0     0   0   0   0   0
3     0   0   0   0   0     0   0   0   0   0
4     0   0   0   0   0     0   0   0   0   0
5     0   0   0   0   0     0   0   0   0   0
6     0   0   0   0   0     0   0   0   0   1
7     0   0   0   0   0     0   0   0   0   0
8     0   0   0   0   0     0   0   0   0   0
9     0   0   0   0   0     0   0   0   1   1
10    0   0   0   0   0     0   0   1   0   0
Shape of y = (112 rows, 80 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  柴胡  蟬蛻
1    0   0   0   0   0   0   0   0   0   0
2    0   0   0   0   0   0   0   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0  

In [69]:
print("shape of X_np", X_np.shape)
print("shape of y_np", y_np.shape)


shape of X_np (112, 101)
shape of y_np (80, 112)


In [83]:
model = Sequential([
    Dense (units=160, input_shape=(101,), activation='relu'),
    Dense (units=2, activation='softmax')
])
model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 160)               16320     
                                                                 
 dense_29 (Dense)            (None, 2)                 322       
                                                                 
Total params: 16,642
Trainable params: 16,642
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [86]:
model.fit(x=X_np, y=y_np,
          batch_size=112, epochs=10, shuffle=True, verbose=2)

# looks like label have to be column vector...

Epoch 1/10


ValueError: in user code:

    File "c:\Users\taliah\miniconda3\envs\ml2\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\taliah\miniconda3\envs\ml2\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\taliah\miniconda3\envs\ml2\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\taliah\miniconda3\envs\ml2\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\taliah\miniconda3\envs\ml2\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\taliah\miniconda3\envs\ml2\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
   